##Text Preprocessing

In [34]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Define the text (FAQs)
faqs = """
Effective warehouse management ensures timely and accurate order fulfillment, reduces operational costs, and improves customer satisfaction.
What are the key functions of a Warehouse Management System (WMS)?

Inventory tracking, order picking, shipping and receiving, labor management, and reporting.
Inventory Management
What is inventory management?

Inventory management involves tracking and controlling stock levels to ensure the right amount of goods is available to meet demand.
How can inventory accuracy be improved?

Implementing barcode scanning, conducting regular cycle counts, and using RFID technology.
What is a cycle count?

A cycle count is a periodic inventory auditing procedure that checks the accuracy of inventory levels without the need for a full inventory count.
Order Fulfillment
What is order fulfillment?

Order fulfillment is the process of receiving, processing, and delivering orders to customers.
What are the common order picking methods?

Single order picking, batch picking, zone picking, and wave picking.
How can order picking efficiency be improved?

By optimizing warehouse layout, using automated picking systems, and implementing pick-to-light or voice picking technologies.
Shipping and Receiving
What is the receiving process in a warehouse?

The receiving process involves unloading goods from delivery vehicles, inspecting the goods, and updating inventory records.
How can the shipping process be optimized?

By using automated shipping software, improving packaging processes, and ensuring accurate labeling and documentation.
What is cross-docking?

Cross-docking is a practice where incoming shipments are directly transferred to outbound trucks with minimal storage time, reducing handling and storage costs.
Labor Management
What is labor management in a warehouse?

Labor management involves planning, scheduling, and monitoring warehouse staff to ensure efficient operations.
How can labor productivity be increased?

By providing proper training, using performance metrics, and implementing labor management software.
What are common challenges in warehouse labor management?

High turnover rates, managing seasonal demand, and ensuring safety and compliance.
Warehouse Layout and Design
What factors should be considered in warehouse layout design?

Storage capacity, material flow, accessibility, safety regulations, and future scalability.
What is the importance of proper warehouse layout?

A well-designed layout maximizes space utilization, improves workflow efficiency, and enhances safety.
How can warehouse space be optimized?

By using vertical storage solutions, implementing flexible racking systems, and regularly reviewing and reorganizing inventory.
Technology and Automation
What are the benefits of warehouse automation?

Increased efficiency, reduced labor costs, improved accuracy, and enhanced safety.
What types of warehouse automation technologies are available?

Automated storage and retrieval systems (AS/RS), conveyor systems, robotics, and warehouse management software.
How does a Warehouse Management System (WMS) work?

A WMS tracks inventory levels, manages order processing, and provides real-time data on warehouse operations.
Safety and Compliance
What are common warehouse safety practices?

Regular safety training, proper use of personal protective equipment (PPE), and maintaining clear and organized work areas.
How can warehouse compliance be ensured?

By adhering to industry regulations, conducting regular audits, and keeping accurate records.
What are the most common warehouse hazards?

Slips and falls, forklift accidents, and improper lifting techniques.
"""

# Tokenize the text into sentences and then into words
sentences = faqs.lower().split('\n')
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
total_words = len(tokenizer.word_index) + 1  # Adding 1 for out of vocabulary (OOV) token

# Create input sequences using tokenizer
input_sequences = []
for line in sentences:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences to ensure uniform length for model input
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')

# Create predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]

# Convert label to categorical
label = tf.keras.utils.to_categorical(label, num_classes=total_words)


##Define and Train the LSTM Model

In [35]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Define the model
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print model summary
print(model.summary())

# Train the model
history = model.fit(predictors, label, epochs=100, verbose=1)


Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 23, 100)           21300     
                                                                 
 lstm_14 (LSTM)              (None, 150)               150600    
                                                                 
 dense_2 (Dense)             (None, 213)               32163     
                                                                 
Total params: 204063 (797.12 KB)
Trainable params: 204063 (797.12 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/100
14/14 [==============================] - 4s 57ms/step - loss: 5.3370 - accuracy: 0.0497
Epoch 2/100
14/14 [==============================] - 1s 57ms/step - loss: 5.0795 - accuracy: 0.0767
Epoch 3/100
14/14 [==============================] - 1s 55ms/step - loss:

# Generating Predictions

In [ ]:
# Example usage
seed_text = "what is order fulfillment"
next_words = 5
generated_text = generate_text(seed_text, next_words)
print(generated_text)